In [50]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

In [52]:
data_dir  = r'.\archive\TrashType_Image_Dataset';

classes = os.listdir(data_dir)
print(classes)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [54]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = ImageFolder(data_dir, transform = transformations)

In [56]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

In [58]:
random_seed = 42
torch.manual_seed(random_seed)

In [60]:
len(dataset)

2528

In [62]:
from torch.utils.data.dataloader import DataLoader
batch_size = 32

In [64]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [66]:
device = get_default_device()
device

device(type='cpu')

In [68]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch {}: train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch+1, result['train_loss'], result['val_loss'], result['val_acc']))
class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet50(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, len(dataset.classes))
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))

model = torch.load("GarbageClassifier.pth")

In [70]:
to_device(model, device)

ResNet(
  (network): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [72]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    prob, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return dataset.classes[preds[0].item()]

In [74]:
loaded_model = model

In [76]:
from PIL import Image
from pathlib import Path

def predict_external_image(image_name):
    image = Image.open(Path(image_name))

    example_image = transformations(image)
    plt.imshow(example_image.permute(1, 2, 0))
    return predict_image(example_image, loaded_model);

In [78]:
classes

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [80]:
classesData = {
    'cardboard':"",
    'glass': "Glass recycling is the processing of waste glass into usable products. Glass that is crushed and ready to be remelted is called cullet. There are two types of cullet: internal and external. Internal cullet is composed of defective products detected and rejected by a quality control process during the industrial process of glass manufacturing, transition phases of product changes (such as thickness and colour changes) and production offcuts. External cullet is waste glass that has been collected or reprocessed with the purpose of recycling. External cullet (which can be pre- or post-consumer) is classified as waste. The word cullet, when used in the context of end-of-waste, will always refer to external cullet. To be recycled, glass waste needs to be purified and cleaned of contamination. Then, depending on the end use and local processing capabilities, it might also have to be separated into different colors. Many recyclers collect different colors of glass separately since glass retains its color after recycling.",
    'metal':"",
    'paper':"The recycling of paper is the process by which waste paper is turned into new paper products. It has a number of important benefits: It saves waste paper from occupying homes of people and producing methane as it breaks down. Because paper fibre contains carbon (originally absorbed by the tree from which it was produced), recycling keeps the carbon locked up for longer and out of the atmosphere. Around two-thirds of all paper products in the US are now recovered and recycled, although it does not all become new paper. After repeated processing, the fibres become too short for the production of new paper - this is why virgin fibre (from sustainably farmed trees) is frequently added to the pulp recipe. Paper recycling pertains to the processes of reprocessing waste paper for reuse. Waste papers are either obtained from paper mill paper scraps, discarded paper materials, and waste paper material discarded after consumer use. Examples of the commonly known papers recycled are old newspapers and magazines.",
    'plastic':"Plastic recycling is the process of recovering scrap or waste plastic and reprocessing the material into useful products. Due to purposefully misleading symbols on plastic packaging and numerous technical hurdles, less than 10% of plastic has ever been recycled. Compared with the lucrative recycling of metal, and similar to the low value of glass recycling, plastic polymers recycling is often more challenging because of low density and low value. Materials recovery facilities are responsible for sorting and processing plastics. As of 2019, due to limitations in their economic viability, these facilities have struggled to make a meaningful contribution to the plastic supply chain. The plastics industry has known since at least the 1970s that recycling of most plastics is unlikely because of these limitations. However, the industry has lobbied for the expansion of recycling while these companies have continued to increase the amount of virgin plastic being produced.",
    'trash':"",
}

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os

def create_app():
    def load_image():
        # Open a file dialog to select an image file
        file_path = filedialog.askopenfilename(
            title="Select an Image",
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp;*.gif")]
        )
        if file_path:
            try:
                # Load the selected image
                img = Image.open(file_path)
                photo = ImageTk.PhotoImage(img)
                
                # Update the label with the new image
                image_label.config(image=photo)
                image_label.image = photo  # Keep a reference to avoid garbage collection
                
                # Pack the image label
                image_label.pack(pady=(0, 5))
                
                imgtype = predict_external_image(file_path)
                
                # Update the text label
                text_label.config(text="Waste Classified as \"" + imgtype + "\" Material")
                text_label.pack(pady=(0, 10))  # Add some padding around the text

                # Update the data label to display the string stored in 'data'
                # Example string in 'data'
                data = classesData[imgtype]
                data_label.config(text=data)
                data_label.pack(pady=(0, 10))  # Add some padding around the data label
            except Exception as e:
                print(f"An error occurred: {e}")
    
    # Create the main window
    root = tk.Tk()
    root.title("Image Display")
    
    # Create a button to load the image
    load_button = tk.Button(root, text="Load Image", command=load_image)
    load_button.pack(pady=10)  # Add some padding around the button
    
    # Create a label to display the image
    image_label = tk.Label(root)
    image_label.pack(pady=(0, 5))  # Add 5 lines of space below the image

    # Create a label to display the text "This image is:"
    text_label = tk.Label(root, text="", font=("Helvetica", 16, "bold"))
    text_label.pack(pady=(0, 10))  # Add some padding around the text

    # Create a label to display the string stored in 'data' with word wrapping
    data_label = tk.Label(root, text="", font=("Helvetica", 14), wraplength=1000, justify="left")
    data_label.pack(pady=(0, 10))  # Add some padding around the data label

    # Run the Tkinter event loop
    root.mainloop()

# Call the function to create the app
create_app()


In [47]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import torchvision.transforms as T


def preprocess(image_path):
    transform = T.Compose([
        T.Resize((256, 256)),  # Resize to match your model input size
        T.ToTensor(),          # Convert to tensor
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)  # Add batch dimension

def segment_objects(image_path):
    image_tensor = preprocess(image_path)
    
    with torch.no_grad():
        output = model(image_tensor)
    
    output = output.squeeze().cpu().numpy()  # Remove batch dimension
    if len(output.shape) == 3:  # Handle multi-class output
        segmentation_mask = np.argmax(output, axis=0)  # Get the class with highest probability for each pixel
    else:
        segmentation_mask = (output > 0.5).astype(int)  # For binary output
    
    return segmentation_mask

def get_color_map():
    # Define a color map for each class
    return np.array([
        [0, 0, 0],        # black for 'cardboard'
        [0, 255, 0],      # green for 'glass'
        [255, 0, 0],      # red for 'metal'
        [0, 0, 255],      # blue for 'paper'
        [255, 255, 0],    # yellow for 'plastic'
        [255, 0, 255]     # magenta for 'trash'
    ], dtype=np.uint8)  # Ensure colors are in uint8 format

def get_labels():
    return ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

def overlay_labels(image_path, mask):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    color_map = get_color_map()
    labels = get_labels()

    try:
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        font = ImageFont.load_default()

    for i in range(6):  # Assuming 6 classes
        mask_class = mask == i
        if np.any(mask_class):
            coords = np.argwhere(mask_class)
            print(f"Coordinates for class {i}: {coords}")  # Debug output
            
            if coords.shape[0] == 0:
                print(f"No coordinates found for class {i}")
                continue

            # Ensure coords is in expected format
            if coords.shape[1] != 2:
                coords = coords.reshape(-1, 2)  # Ensure 2D shape if possible

            min_x, min_y = coords.min(axis=0)
            max_x, max_y = coords.max(axis=0)
            
            # Ensure valid coordinates
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)

            # Draw bounding box and text
            draw.rectangle([min_y, min_x, max_y, max_x], outline=tuple(color_map[i]), width=2)
            draw.text((min_y, min_x), labels[i], fill=tuple(color_map[i]), font=font)

    return image

# Example usage
image_path = '.\\test.jpg'
segmentation_mask = segment_objects(image_path)
annotated_image = overlay_labels(image_path, segmentation_mask)
annotated_image.show()  # Show the annotated image


Coordinates for class 0: [[0]
 [1]
 [2]
 [4]
 [5]]


ValueError: cannot reshape array of size 5 into shape (2)

In [39]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Load your GarbageClassifier model
model = torch.load('GarbageClassifier.pth')
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define classes for your GarbageClassifier
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Load pre-trained Faster R-CNN model for object detection
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()
detection_model.to(device)

# Define preprocessing for GarbageClassifier
def preprocess(image):
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
    ])
    return transform(image)

# Function to classify objects
def predict_image(img, model):
    xb = img.unsqueeze(0).to(device)  # Convert to batch of 1 and move to device
    with torch.no_grad():
        yb = model(xb)
    prob, preds = torch.max(yb[0], dim=1)  # Get the predicted class
    return classes[preds[0].item()]

# Process image using Faster R-CNN and GarbageClassifier
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    color_map = np.array([
        [0, 0, 0],        # black for 'cardboard'
        [0, 255, 0],      # green for 'glass'
        [255, 0, 0],      # red for 'metal'
        [0, 0, 255],      # blue for 'paper'
        [255, 255, 0],    # yellow for 'plastic'
        [255, 0, 255]     # magenta for 'trash'
    ], dtype=np.uint8)

    font = ImageFont.load_default()

    # Preprocess image for object detection
    transform = T.Compose([
        T.ToTensor(),
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        detections = detection_model(image_tensor)[0]

    # Draw bounding boxes and labels
    for i, (box, label, score) in enumerate(zip(detections['boxes'], detections['labels'], detections['scores'])):
        if score > 0.5:  # Confidence threshold
            min_x, min_y, max_x, max_y = box.int().cpu().numpy()

            # Debugging: Print the coordinates and class
            print(f"Object {i}: Bounding box: ({min_x}, {min_y}), ({max_x}, {max_y})")

            # Extract the object region
            obj_region = image_tensor[:, min_y:max_y, min_x:max_x]
            if obj_region.shape[1] == 0 or obj_region.shape[2] == 0:
                continue  # Skip empty regions

            # Classify the object
            obj_label = predict_image(obj_region.squeeze(0), model)
            print(f"Object {i}: Classified as {obj_label}")

            # Draw bounding box and label
            draw.rectangle([min_x, min_y, max_x, max_y], outline=tuple(color_map[classes.index(obj_label)]), width=2)
            draw.text((min_x, min_y), obj_label, fill=tuple(color_map[classes.index(obj_label)]), font=font)

    return image

# Example usage
image_path = '.\\test.jpg'
annotated_image = process_image(image_path)
annotated_image.show()  # Show the annotated image

C:\Users\sahan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sahan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Object 0: Bounding box: (721, 309), (811, 658)
Object 1: Bounding box: (837, 338), (907, 656)
Object 2: Bounding box: (438, 328), (548, 684)
Object 3: Bounding box: (766, 335), (860, 675)
Object 4: Bounding box: (65, 451), (354, 644)
Object 5: Bounding box: (842, 339), (902, 494)
Object 6: Bounding box: (930, 550), (1136, 677)
Object 7: Bounding box: (806, 335), (876, 666)
Object 8: Bounding box: (0, 475), (1187, 755)
Object 9: Bounding box: (56, 525), (341, 617)
Object 10: Bounding box: (53, 592), (321, 649)
Object 11: Bounding box: (61, 575), (345, 631)
Object 12: Bounding box: (771, 555), (861, 678)
Object 13: Bounding box: (56, 558), (354, 678)


In [47]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Load your GarbageClassifier model
model = torch.load('GarbageClassifier.pth')
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define classes for your GarbageClassifier
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Load pre-trained Faster R-CNN model for object detection
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()
detection_model.to(device)

# Define preprocessing for GarbageClassifier
def preprocess(image):
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
    ])
    return transform(image)

# Function to classify objects
def predict_image(img, model):
    xb = img.unsqueeze(0).to(device)  # Convert to batch of 1 and move to device
    with torch.no_grad():
        yb = model(xb)
    prob, preds = torch.max(yb[0], dim=1)  # Get the predicted class
    return classes[preds[0].item()]

# Process image using Faster R-CNN and GarbageClassifier
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    color_map = np.array([
        [0, 0, 0],        # black for 'cardboard'
        [0, 255, 0],      # green for 'glass'
        [255, 0, 0],      # red for 'metal'
        [0, 0, 255],      # blue for 'paper'
        [255, 255, 0],    # yellow for 'plastic'
        [255, 0, 255]     # magenta for 'trash'
    ], dtype=np.uint8)

    font = ImageFont.load_default()

    # Preprocess image for object detection
    transform = T.Compose([
        T.ToTensor(),
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        detections = detection_model(image_tensor)[0]

    # Debugging: Print total detected objects and their scores
    print(f"Detected objects: {len(detections['boxes'])}")
    for i, (box, label, score) in enumerate(zip(detections['boxes'], detections['labels'], detections['scores'])):
        print(f"Object {i}: Bounding box: {box}, Score: {score}")

    # Draw bounding boxes and labels
    for i, (box, label, score) in enumerate(zip(detections['boxes'], detections['labels'], detections['scores'])):
        if score > 0.5:  # Confidence threshold
            min_x, min_y, max_x, max_y = box.int().cpu().numpy()

            # Debugging: Print bounding box details
            print(f"Drawing bounding box: ({min_x}, {min_y}), ({max_x}, {max_y})")

            # Extract the object region
            obj_region = image_tensor[:, min_y:max_y, min_x:max_x]
            if obj_region.shape[1] == 0 or obj_region.shape[2] == 0:
                continue  # Skip empty regions

            # Resize object region to match classifier input size
            obj_region = T.Resize((256, 256))(obj_region.squeeze(0))  # Resize to classifier input size

            # Classify the object
            obj_label = predict_image(obj_region.unsqueeze(0), model)
            print(f"Object {i}: Classified as {obj_label}")

            # Draw bounding box and label
            draw.rectangle([min_x, min_y, max_x, max_y], outline=tuple(color_map[classes.index(obj_label)]), width=2)
            draw.text((min_x, min_y - 10), obj_label, fill=tuple(color_map[classes.index(obj_label)]), font=font)

    return image

# Example usage
image_path = '.\\test.jpg'
annotated_image = process_image(image_path)
annotated_image.show()  # Show the annotated image
annotated_image.save('annotated_image_with_labels.jpg')  # Save the annotated image


C:\Users\sasan\AppData\Local\Temp\ipykernel_1416\1440616192.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('GarbageClassifier.pth')


Detected objects: 45
Object 0: Bounding box: tensor([721.3423, 309.4503, 812.0912, 658.3715]), Score: 0.9944170713424683
Object 1: Bounding box: tensor([837.5850, 338.4106, 907.4632, 657.1381]), Score: 0.9919330477714539
Object 2: Bounding box: tensor([439.0090, 328.3222, 549.0280, 683.6404]), Score: 0.977828860282898
Object 3: Bounding box: tensor([766.8976, 337.0799, 860.4631, 675.6911]), Score: 0.961835503578186
Object 4: Bounding box: tensor([ 64.9997, 451.3032, 354.7159, 643.7771]), Score: 0.9197129607200623
Object 5: Bounding box: tensor([842.0444, 339.2059, 902.1689, 494.5328]), Score: 0.9180073738098145
Object 6: Bounding box: tensor([ 930.7585,  548.6348, 1136.9679,  677.1661]), Score: 0.8908308744430542
Object 7: Bounding box: tensor([806.6801, 335.7841, 876.8629, 666.4393]), Score: 0.8302330374717712
Object 8: Bounding box: tensor([   0.0000,  475.5961, 1187.0000,  755.3837]), Score: 0.7574615478515625
Object 9: Bounding box: tensor([ 56.7125, 525.4404, 341.2184, 617.7027]),

In [48]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw
import torchvision.transforms as T

# Load pre-trained Faster R-CNN model for object detection
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
detection_model.to(device)

# Function to process image and draw bounding boxes
def process_image(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    
    transform = T.Compose([
        T.ToTensor(),
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Perform object detection
    with torch.no_grad():
        detections = detection_model(image_tensor)[0]

    # Draw bounding boxes
    for i, (box, score) in enumerate(zip(detections['boxes'], detections['scores'])):
        if score > 0.5:  # Confidence threshold
            min_x, min_y, max_x, max_y = box.int().cpu().numpy()
            
            # Draw bounding box
            draw.rectangle([min_x, min_y, max_x, max_y], outline="black", width=2)
    
    return image

# Example usage
image_path = '.\\test.jpg'
annotated_image = process_image(image_path)
annotated_image.show()  # Show the annotated image
annotated_image.save('gimage.jpg')  # Save the annotated image


In [57]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw, ImageFont
import torchvision.transforms as T
import numpy as np
import cv2
from io import BytesIO

# Load your GarbageClassifier model
garbage_model = torch.load('GarbageClassifier.pth')
garbage_model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
garbage_model.to(device)

# Define classes for your GarbageClassifier
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# Load pre-trained Faster R-CNN model for object detection
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()
detection_model.to(device)

# Define preprocessing for GarbageClassifier
def preprocess_image(image):
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
    ])
    return transform(image)

# Function to classify objects
def predict_image(img, model):
    xb = img.unsqueeze(0).to(device)  # Convert to batch of 1 and move to device
    with torch.no_grad():
        yb = model(xb)
    prob, preds = torch.max(yb[0], dim=1)  # Get the predicted class
    return classes[preds[0].item()]

# Function to extract and classify object regions
def process_image(image_path, model, classes, device):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    width, height = image.size

    # Define transforms
    preprocess = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
    ])

    # Dummy function for object detection
    def dummy_detect_objects(image):
        # Replace this with your actual object detection code
        return [(50, 50, 150, 150), (200, 200, 300, 300)]  # Example bounding boxes

    # Detect objects in the image
    detections = dummy_detect_objects(image)

    # Prepare to draw bounding boxes and labels
    draw = ImageDraw.Draw(image)

    # Classification transform
    classify_transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
    ])

    for i, (min_x, min_y, max_x, max_y) in enumerate(detections):
        # Crop the object region from the image
        obj_region = image.crop((min_x, min_y, max_x, max_y))
        obj_region = classify_transform(obj_region).unsqueeze(0).to(device)

        # Classify the object
        model.eval()
        with torch.no_grad():
            outputs = model(obj_region)
            _, preds = torch.max(outputs, 1)
            label = classes[preds.item()]

        # Draw bounding box and label
        draw.rectangle([min_x, min_y, max_x, max_y], outline="black", width=2)
        draw.text((min_x, min_y), label, fill="black")

    return image

# Example usage
image_path = '.\\test.jpg'
annotated_image = process_image(image_path, garbage_model, classes, device)
annotated_image.show()  # Show the annotated image
annotated_image.save('annotated_image_with_labels.jpg')  # Save the annotated image


C:\Users\sasan\AppData\Local\Temp\ipykernel_1416\1070472941.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  garbage_model = torch.load('GarbageClassifier.pth')


In [85]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision import transforms as T
from PIL import Image, ImageDraw, ImageFont
import numpy as np


def to_device(tensor, device):
    return tensor.to(device)

# Initialize your model and device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
garbage_model = torch.load("GarbageClassifier.pth").to(device)  # Load your trained model
garbage_model.eval()

# Load a pre-trained Faster R-CNN model for object detection
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.to(device)
detection_model.eval()

def detect_objects(image, model, device):
    image_tensor = F.to_tensor(image).unsqueeze(0).to(device)
    with torch.no_grad():
        predictions = model(image_tensor)
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    threshold = 0.5
    boxes = boxes[scores > threshold]
    return [tuple(box) for box in boxes]

def preprocess_image(image):
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor()
    ])
    return transform(image)

def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    with torch.no_grad():
        yb = model(xb)
        prob, preds = torch.max(yb, dim=1)
    return classes[preds[0].item()]

def process_image(image_path, model, classes, device):
    image = Image.open(image_path).convert("RGB")
    detections = detect_objects(image, detection_model, device)
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    for min_x, min_y, max_x, max_y in detections:
        obj_region = image.crop((min_x, min_y, max_x, max_y))
        obj_region_preprocessed = preprocess_image(obj_region)
        label = predict_image(obj_region_preprocessed, model)

        draw.rectangle([min_x, min_y, max_x, max_y], outline="black", width=2)
        draw.text((min_x, min_y - 20), label, fill="black", font=font)

    return image

# Example usage
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']  # Define your classes
image_path = '.\\cimage.jpg'
annotated_image = process_image(image_path, garbage_model, classes, device)
annotated_image.show()  # Show the annotated image
annotated_image.save('annotated_image_with_labels.jpg')  # Save the annotated image
